<a href="https://colab.research.google.com/github/mansaeng/Machine-Learning/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 모듈 Importing
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (
    ensemble,
    preprocessing,
    tree,
)
from sklearn.metrics import (
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
)
from yellowbrick.classifier import (
    ConfusionMatrix,
    ROCAUC,
)
from yellowbrick.model_selection import (
    LearningCurve,
)

# 판다스는 데이터를 가공 처리하는 모듈이다.
# 사이킷런은 예측 모델링을 제공한다.
# 옐로우브릭은 모델 검증을 위한 시각화 라이브러리이다.


# from ~ import ~ 형식으로 Importing 할 때 (
#   이렇게,
#   Pythonic한 방법을,
#   이제 알았다.
# )

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


# *from pandas import **

별표를 사용하여 라이브러리를 불러들이면
명시적이지 않으므로 코드 이해가 어려워진다.
지정해서 불러오는 습관을 들이자.

In [2]:
# 머신러닝에 필요한 데이터 가져오기
url = (
    "https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.xls"
)
df = pd.read_excel(url) # 판다스의 read_excel은 파일 뿐만이 아니라 url의 데이터도 가져올 수 있다.
orig_df = df

**주어진 데이터 :**
> 타이타닉호 탑승객 데이터

**목표 :**
> 해당 탑승객이 생존했는지 사망했는지 예측(분류)

In [3]:
# dtypes로 데이터 타입을 확인할 수 있다.
# 데이터 타입을 분류할 수 없다고 판단되면 object로 분류된다.
df.dtypes

pclass         int64
survived       int64
name          object
sex           object
age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home.dest     object
dtype: object

In [4]:
# .shape로 행과 열의 개수를 알 수 있다.
df.shape

(1309, 14)

In [5]:
# .describe()로 데이터의 통계적 요약을 알 수 있다.
# 이것을 이용하면 데이터셋에서 이용해야 할 데이터가 무엇인지 어떻게 이용할 지 대략적으로 알 수 있을 것 같다.
# count값을 확인하면 누락된 데이터를 알 수 있다.
# min과 max값을 이상치의 유무를 확인할 수 있다.
df.describe()

,pclass,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


In [6]:
# .isnull().sum() 으로 데이터의 누락수를 알 수 있다.
df.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [7]:
# value_counts로 원하는 데이터의 값의 수를 확인할 수 있다.
# null 또는 NaN 값을 포함하고 싶다면 dropna=False 옵션을 추가해주면 된다.
df.sex.value_counts(dropna=False)

male      843
female    466
Name: sex, dtype: int64

In [8]:
# 누락된 데이터를 확인할 수 있다.
# 이렇게 범주형이므로 누락된 값을 더미 칼럼으로 표시해 볼 수도 있겠다.
df.embarked.value_counts(dropna=False)

S      914
C      270
Q      123
NaN      2
Name: embarked, dtype: int64

In [9]:
name = df.name
name.head(3)

# pandas의 .drop 메서드는 행 또는 열 제거를 할 때 사용한다.
# 승객의 생존여부에 관계가 없는 이름, 티켓, 목적지, 몇 번 보트를 탔는지, 사체번호, 방번호는 제외한다.
df = df.drop(
    columns=[
        'name',
        'ticket',
        'home.dest',
        'boat',
        'body',
        'cabin'
    ]
)

In [10]:
# 현재 데이터프레임의 상태이다.
print(df)

      pclass  survived     sex      age  sibsp  parch      fare embarked
0          1         1  female  29.0000      0      0  211.3375        S
1          1         1    male   0.9167      1      2  151.5500        S
2          1         0  female   2.0000      1      2  151.5500        S
3          1         0    male  30.0000      1      2  151.5500        S
4          1         0  female  25.0000      1      2  151.5500        S
...      ...       ...     ...      ...    ...    ...       ...      ...
1304       3         0  female  14.5000      1      0   14.4542        C
1305       3         0  female      NaN      1      0   14.4542        C
1306       3         0    male  26.5000      0      0    7.2250        C
1307       3         0    male  27.0000      0      0    7.2250        C
1308       3         0    male  29.0000      0      0    7.8750        S

[1309 rows x 8 columns]


In [11]:
# get_dummies를 이용하면 역상관관계 또는 완벽환 상관관계, 매우 높거나 낮은 양의 상관관계의 데이터를 나눠준다.
df = pd.get_dummies(df)
df.columns

# sex를 female, male로 분류하고
# embarked를 C, Q, S로 분류해준다.

Index(['pclass', 'survived', 'age', 'sibsp', 'parch', 'fare', 'sex_female',
       'sex_male', 'embarked_C', 'embarked_Q', 'embarked_S'],
      dtype='object')

In [12]:
# 위에서 설명했듯이 생존여부를 예측하기 위해서 생존여부 데이터를 drop으로 제거한다.
y = df.survived
X = df.drop(columns="survived")

In [13]:
### 항상 학습과 검증에는 서로 다른 데이터가 사용되어야 한다.(모델이 얼마나 잘 일반화 되는지 알 수 없기 때문)
### 사이킷런으로 준비된 데이터의 30%를 분리하여 검증을 위한 데이터로 빼둘 것이다.
### random_state는 무작위 값을 생성하는 시드를 고정한다. 같은 무작위 값이 출력된다.(다른 모델과 성능 비교 시 공정함을 위해)
import sklearn

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, test_size = 0.3, random_state=42
)

In [14]:
### 이 부분은 잘 이해하지 못했다.
# age 열은 누락된 값을 가지고 있기 때문에, 이들을 수치형 값으로 대치해야 한다.
# 우선 대치자로 학습용 데이터셋만 대치를 진행한다.
# 그리고 사용된 같은 대치자로 검증용 데이터셋의 대치도 진행한다.
# 이 순서를 따르지 않으면 미래의 데이터(검증용 데이터셋)의 정보가 과거의 데이터(학습용 데이터)로 누수되는 문제가 발생할 수 있다.
# 대치 과정에서 대치자는 무엇을 어떻게 대치했는지를 학습한다.
# 그러면 학습된 대치자로 테스트용 데이터셋의 누락된 값을 대치할 수 있다.

from sklearn.experimental import (
    enable_iterative_imputer,
)
from sklearn import impute

num_cols = [
      "pclass",
      "age",
      "sibsp",
      "parch",
      "fare",
      "sex_female",
]

imputer = impute.IterativeImputer()
imputed = imputer.fit_transform(
    X_train[num_cols]
)

X_train.loc[:,num_cols] = imputed
imputed = imputer.transform(X_test[num_cols])
X_test.loc[:,num_cols] = imputed

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [15]:
# 만약 중앙값으로 데이터를 대치하고 싶다면 그냥 pandas를 사용하면 된다.

meds = X_train.median()
X_train = X_train.fillna(meds)
X_test = X_test.fillna(meds)